In [1]:
import pandas as pd
import numpy as np

In [2]:
aduanas = pd.read_csv('/home/luis-loreto/Documentos/Trabajo/Balanza2024_ANAM/Lista_de_aduanas_2023.csv', sep = '|')

In [ ]:
def imp_b(num, mes):

       B1 = pd.read_csv(f'/home/luis-loreto/Documentos/Trabajo/Balanza2024_ANAM/{num}{mes}2024/t_551_01.txt', 
                        sep = '|', header=None, usecols=[0, 1, 2, 8, 10, 21], 
                        names=['Patente aduanal', 'Numero de pedimento', 'Seccion aduanera', 'Valor en aduana', 'Valor en dólares', 'Pais de origen'])
    
       B2 = pd.read_csv(f'/home/luis-loreto/Documentos/Trabajo/Balanza2024_ANAM/{num}{mes}2024/t_551_02.txt', 
                        sep = '|', header=None, usecols=[0, 1, 2, 8, 10, 21], 
                        names=['Patente aduanal', 'Numero de pedimento', 'Seccion aduanera', 'Valor en aduana', 'Valor en dólares', 'Pais de origen'])
       
       B = pd.concat([B1, B2], axis=0)
       return B

def imp_a(num, mes):
    A = pd.read_csv(f'/home/luis-loreto/Documentos/Trabajo/Balanza2024_ANAM/{num}{mes}2024/t_501.txt', 
        sep = '|', header=None, usecols=[0, 1, 2, 3, 4], 
        names=['Patente aduanal', 'Numero de pedimento', 'Seccion aduanera', 
               'Tipo de operacion', 'Clave de documento'])
    
    A = A[(A['Tipo de operacion'] == 1) & (A['Clave de documento'] == 'T1')]
    
    return A

def cruce(num, mes):
    A = imp_a(num, mes) 
    B = imp_b(num, mes)
    C = pd.merge(A, B, on = ['Patente aduanal', 'Numero de pedimento', 'Seccion aduanera'], how='left')
    del A, B
    return C

def condiciones(mes):

    mes.dropna(inplace=True)

    conditions = [
        mes['Valor en dólares'] < 1,
        (mes['Valor en dólares'] >= 1) & (mes['Valor en dólares'] < 7),
        (mes['Valor en dólares'] >= 7) & (mes['Valor en dólares'] < 10),
        (mes['Valor en dólares'] >= 10) & (mes['Valor en dólares'] < 20),
        (mes['Valor en dólares'] >= 20) & (mes['Valor en dólares'] < 30),
        (mes['Valor en dólares'] >= 30) & (mes['Valor en dólares'] < 40),
        (mes['Valor en dólares'] >= 40) & (mes['Valor en dólares'] < 50),
        (mes['Valor en dólares'] >= 50) & (mes['Valor en dólares'] < 1000),
        (mes['Valor en dólares'] >= 1000) & (mes['Valor en dólares'] < 2500),
        (mes['Valor en dólares'] >= 2500) & (mes['Valor en dólares'] < 5000),
        mes['Valor en dólares'] >= 5000
    ]
    choices = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']
    mes['Rango USD'] = np.select(conditions, choices)
    return mes

In [ ]:
Enero = cruce("01", "Enero")
Febrero = cruce("02", "Febrero")
Marzo = cruce("03", "Marzo")
Abril = cruce("04", "Abril")
Mayo = cruce("05", "Mayo")
Junio = cruce("06", "Junio")
Julio = cruce("07", "Julio")
Agosto = cruce("08", "Agosto")
Septiembre = cruce("09", "Septiembre")

In [ ]:
T12024 = pd.concat([Enero, Febrero], axis=0)
del Enero, Febrero
T12024 = pd.concat([T12024, Marzo], axis=0)
del Marzo
T12024 = pd.concat([T12024, Abril], axis=0)
del Abril
T12024 = pd.concat([T12024, Mayo], axis=0)
del Mayo
T12024 = pd.concat([T12024, Junio], axis=0)
del Junio
T12024 = pd.concat([T12024, Julio], axis=0)
del Julio
T12024 = pd.concat([T12024, Agosto], axis=0)
del Agosto
T12024 = pd.concat([T12024, Septiembre], axis=0)
del Septiembre

In [18]:
Septiembre['Rango USD'] = np.where(Septiembre['Valor en dólares'] < 1, 1, 0)

In [ ]:
Enero['Rango USD'] = Enero['Valor en dólares'].apply(lambda x: 1 if x < 1 else 0)

In [19]:
Septiembre

,Patente aduanal,Numero de pedimento,Seccion aduanera,Tipo de operacion,Clave de documento,Valor en aduana,Valor en dólares,Pais de origen,Rango USD
0,6087,ec\[XXS,470,1,T1,19090.0,973.80,CHE,0
1,6087,jha`^^],470,1,T1,7802.0,398.00,CHN,0
2,6087,ec\[\YS,470,1,T1,9018.0,455.08,USA,0
3,6087,fd]]XYU,470,1,T1,2276.0,114.83,AUT,0
4,6087,fd]]XYU,470,1,T1,2768.0,139.67,HKG,0
...,...,...,...,...,...,...,...,...,...
10647225,3547,b\[VWUM,470,1,T1,NaN,NaN,NaN,0
10647226,3547,ga`[ZXW,470,1,T1,NaN,NaN,NaN,0
10647227,3547,d^]XXSQ,470,1,T1,NaN,NaN,NaN,0
10647228,3547,ga`[ZZW,470,1,T1,NaN,NaN,NaN,0


In [23]:
def union(num, mes, parte):
    df = pd.read_csv(f'/home/luis-loreto/Documentos/Trabajo/Balanza2024_ANAM/{num}{mes}2024/t_551_{parte}.txt', 
                sep = '|', header=None, usecols=[0, 1, 2, 8, 10, 21], 
                names=['Patente aduanal', 'Numero de pedimento', 'Seccion aduanera', 
                       'Valor en aduana', 'Valor en dólares', 'Pais de origen'])
    return df

In [ ]:
Enero_01 = union("01","Enero", "01")
Enero_02 = union("01","Enero", "02")
Enero_ = pd.concat([Enero_01, Enero_02], axis=0)


In [9]:
merged = pd.merge(Enero, Enero_, on = ['Patente aduanal', 'Numero de pedimento', 'Seccion aduanera'], how='left')

In [10]:
merged

,Patente aduanal,Numero de pedimento,Seccion aduanera,Tipo de operacion,Clave de documento,Valor en aduana,Valor en dólares,Pais de origen
0,1695,f`bYVYU,650,1,T1,24620.0,1443.23,GBR
1,1695,ica\ZY[,650,1,T1,1705.0,100.00,USA
2,1695,ica\ZY[,650,1,T1,6649.0,390.00,CHN
3,1695,ica\ZY[,650,1,T1,6333.0,371.45,USA
4,1695,ica\ZY[,650,1,T1,15948.0,935.43,USA
...,...,...,...,...,...,...,...,...
17920405,973,jc`^[^],71,1,T1,14479.0,840.15,CHN
17920406,973,jc`^[^],71,1,T1,10340.0,600.00,CHN
17920407,973,jc`^[^],71,1,T1,16163.0,937.89,USA
17920408,973,jc`^[^],71,1,T1,8358.0,485.00,CHN


In [2]:
A = pd.read_csv('/home/luis-loreto/Documentos/Trabajo/Balanza2024_ANAM/01Enero2024/t_501.txt', 
                    sep = '|', header=None, usecols=[0, 1, 2, 3, 4], 
                    names=['Patente aduanal', 'Numero de pedimento', 'Seccion aduanera', 
                           'Tipo de operacion', 'Clave de documento'])

In [3]:
A1 = A[(A['Tipo de operacion'] == 1) & (A['Clave de documento'] == 'T1')]

In [4]:
A2 = A1[['Patente aduanal', 'Numero de pedimento', 'Seccion aduanera']]

In [5]:
B1 = pd.read_csv('/home/luis-loreto/Documentos/Trabajo/Balanza2024_ANAM/01Enero2024/t_551_01.txt', 
                    sep='|', header=None, usecols=[0, 1, 2, 8, 10, 21], 
                    names=['Patente aduanal', 'Numero de pedimento', 'Seccion aduanera', 
                           'Valor en aduana', 'Valor en dolares', 'Pais de origen'])

In [6]:
B2 = pd.read_csv('/home/luis-loreto/Documentos/Trabajo/Balanza2024_ANAM/01Enero2024/t_551_02.txt', 
                    sep='|', header=None, usecols=[0, 1, 2, 8, 10, 21], 
                    names=['Patente aduanal', 'Numero de pedimento', 'Seccion aduanera', 
                           'Valor en aduana', 'Valor en dolares', 'Pais de origen'])

In [7]:
B = pd.concat([B1, B2], axis=0)

In [10]:
merged = pd.merge(A1, B, on = ['Patente aduanal', 'Numero de pedimento', 'Seccion aduanera'], how='inner')

In [11]:
merged

,Patente aduanal,Numero de pedimento,Seccion aduanera,Tipo de operacion,Clave de documento,Valor en aduana,Valor en dolares,Pais de origen
0,1695,f`bYVYU,650,1,T1,24620.0,1443.23,GBR
1,1695,ica\ZY[,650,1,T1,1705.0,100.00,USA
2,1695,ica\ZY[,650,1,T1,6649.0,390.00,CHN
3,1695,ica\ZY[,650,1,T1,6333.0,371.45,USA
4,1695,ica\ZY[,650,1,T1,15948.0,935.43,USA
...,...,...,...,...,...,...,...,...
17920405,973,jc`^[^],71,1,T1,14479.0,840.15,CHN
17920406,973,jc`^[^],71,1,T1,10340.0,600.00,CHN
17920407,973,jc`^[^],71,1,T1,16163.0,937.89,USA
17920408,973,jc`^[^],71,1,T1,8358.0,485.00,CHN
